# INSTALACIÓN DE BIBLIOTECA SELENIUM PARA SCRAPING

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


# FUNCIONES QUE UTILIZA EL PROGRAMA DE SCRAPING

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException

In [ ]:

def configurar_driver():
    """
    Configura y retorna un objeto driver de Selenium con las opciones necesarias para su ejecución.

    Las opciones configuradas permiten que el navegador se ejecute en modo headless (sin interfaz gráfica),
    lo cual es útil para ejecuciones en servidores o entornos donde no se requiere la interfaz gráfica del navegador.
    También se configuran opciones para evitar problemas comunes en entornos de contenedores como Docker.

    Returns:
        webdriver.Chrome: Una instancia del driver de Chrome configurada para ser usada por las funciones de scraping.
    """
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = "/usr/bin/chromium-browser"

    return webdriver.Chrome(options=options)

def extraer_datos_libro(driver, url):
    """
    Extrae datos de un libro desde una página web usando Selenium.

    Args:
        driver (selenium.webdriver): El driver de Selenium WebDriver que controla el navegador.
        url (str): La URL de la página web del libro.

    Returns:
        dict: Un diccionario con los siguientes datos del libro:
            - "Title" (str): El título del libro.
            - "Authors" (str): Una cadena con los nombres de los autores separados por comas.
            - "Genres" (str): Una cadena con los géneros del libro separados por comas.
            - "Synopsis" (str): La sinopsis del libro.
            - "URL" (str): La URL de la página web del libro.
    """
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#title h1")))

    # Extraer título
    title = driver.find_element(By.CSS_SELECTOR, "#title h1").text

    # Extraer autor
    author_elements = driver.find_elements(By.CSS_SELECTOR, "#autor a.dinSource")
    authors = [author.text for author in author_elements]

    # Extraer género
    genre_elements = driver.find_elements(By.CSS_SELECTOR, "#genero a.dinSource")
    genres = [genre.text for genre in genre_elements]

    # Extraer sinopsis
    synopsis = driver.find_element(By.CSS_SELECTOR, "#sinopsis span").text

    return {
        "Title": title,
        "Authors": ", ".join(authors),
        "Genres": ", ".join(genres),
        "Synopsis": synopsis,
        "URL": url
    }

#CÓDIGO DEL PROGRAMA DE SCRAPING

In [ ]:
import pandas as pd

# Configurar el driver
driver = configurar_driver()
url = "https://ww3.lectulandia.com/"

# Inicializar lista para almacenar datos de libros
books_data = []

try:
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.taxType[data='genero']"))
    ).click()

    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "#secgenero ul li a.term"))
    )

    generos = driver.find_elements(By.CSS_SELECTOR, "#secgenero ul li a.term")
    nombres_generos = [genero.text for genero in generos]  # Recolectar los nombres antes de cualquier otra interacción
    generos_procesados = 0  # Contador de géneros procesados

    for i, nombre_genero in enumerate(nombres_generos):
        if generos_procesados >= 10:
            break  # Detener el bucle después de procesar 10 géneros

        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(generos[i])
            )
            generos[i].click()  # Utilizar click directo en vez de execute_script
            WebDriverWait(driver, 10).until(lambda driver: len(driver.window_handles) > 1)
            new_window = driver.window_handles[-1]
            driver.switch_to.window(new_window)

            if WebDriverWait(driver, 10).until(lambda _: len(driver.find_elements(By.CSS_SELECTOR, ".page-nav .page-numbers")) > 2):
                last_page_number = driver.find_elements(By.CSS_SELECTOR, ".page-nav .page-numbers")[-2].text

                if int(last_page_number) >= 10:
                    print(f"El género '{nombre_genero}' tiene más de a 10 pestañas")
                    generos_procesados += 1  # Incrementar el contador de géneros procesados

                    for _ in range(10):
                        books = WebDriverWait(driver, 10).until(
                            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card-click-target[tabindex='-1']"))
                        )
                        print(f"Cantidad de elementos encontrados en la página actual: {len(books)}")


                        for book in books:
                            href = book.get_attribute('href')
                            print(f"Libro encontrado de género '{nombre_genero}' con enlace: {href}")
                            driver.execute_script("window.open(arguments[0]);", href)
                            driver.switch_to.window(driver.window_handles[-1])
                            book_data = extraer_datos_libro(driver, href)
                            books_data.append(book_data)
                            driver.close()
                            driver.switch_to.window(new_window)

                        # Re-localizar el navigation_block antes de buscar el botón "Siguiente"
                        navigation_block = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, ".page-nav"))
                        )

                        try:
                            next_button = navigation_block.find_element(By.CSS_SELECTOR, "a.next.page-numbers")
                            driver.execute_script("arguments[0].click();", next_button)
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card-click-target[tabindex='-1']"))
                            )
                        except NoSuchElementException:
                            print("No se encontró el botón 'Siguiente'. Fin del proceso de navegación.")
                            break
        except (NoSuchElementException, TimeoutException):
            print(f"No hay suficientes libros en el género '{nombre_genero}'. Pasando al siguiente género.")
            driver.close()  # Cerrar la pestaña actual
            driver.switch_to.window(driver.window_handles[0])  # Volver a la ventana principal
            continue  # Continúa con el siguiente género

finally:
    driver.quit()

# Convertir los datos de los libros a un DataFrame de Pandas
books_data = pd.DataFrame(books_data)

No hay suficientes libros en el género 'Arqueología'. Pasando al siguiente género.
No hay suficientes libros en el género 'Arquitectura'. Pasando al siguiente género.
El género 'Arte' tiene más de a 10 pestañas
Cantidad de elementos encontrados en la página actual: 24
Libro encontrado de género 'Arte' con enlace: https://ww3.lectulandia.com/book/fabulas-mitologicas-en-espana-i/
Libro encontrado de género 'Arte' con enlace: https://ww3.lectulandia.com/book/la-otra-gioconda/
Libro encontrado de género 'Arte' con enlace: https://ww3.lectulandia.com/book/tiempos-criticos/
Libro encontrado de género 'Arte' con enlace: https://ww3.lectulandia.com/book/biblioteca-studio-ghibli-porco-rosso/
Libro encontrado de género 'Arte' con enlace: https://ww3.lectulandia.com/book/el-temperamento-y-la-naturaleza-escritos-sobre-el-arte/
Libro encontrado de género 'Arte' con enlace: https://ww3.lectulandia.com/book/malas-mujeres/
Libro encontrado de género 'Arte' con enlace: https://ww3.lectulandia.com/book/

# FUNCIÓN DE TRANSFORMACIÓN DE CADENA DE TEXTO EN EMBEDDINGS (UTILIZADA PARA LA RESEÑA DE CADA LIBRO)




In [ ]:
from sentence_transformers import SentenceTransformer

def descripcion_transformer(descripcion):
    """
    Genera un embedding para una descripción utilizando un modelo multilingüe de Sentence Transformers.

    Parámetros:
    descripcion (str): La descripción del libro (o cualquier texto) para la cual se generará el embedding.

    Retorna:
    numpy.ndarray: El embedding de la descripción.
    """
    # Cargar un modelo multilingüe
    model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

    # Generar el embedding para la descripción
    descripcion_embedding = model.encode(descripcion)

    return descripcion_embedding

In [ ]:
books_data["Embebbed_Synopsis"] = [descripcion_transformer(x) for x in books_data["Synopsis"]]

[{'Title': 'Fábulas mitológicas en España I',
  'Authors': 'José María de Cossío',
  'Genres': 'Arte, Ciencias sociales, Divulgación, Ensayo',
  'Synopsis': 'En las «Fábulas mitológicas de España», José María de Cossío planteó una nueva visión sobre nuestra literatura al sacar a la luz sus aspectos antirrealistas y universales y, por tanto, los menos admitidos cuando esta obra se publicó por primera vez en 1952. Al ofrecer al lector la reedición de este libro, agotado desde hacía mucho tiempo, lo hacemos no sólo con la intencion de recuperar un «clásico» en su especialidad, sino que tenemos el convencimiento de que se trata de una obra plenamente actual, que no ha perdido un ápice de interés o utilidad pues, desde su primera salida al mercado, no ha aparecido ningún otro trabajo que, como éste, rastreé totalmente la presencia de la mitología en la cultura literaria española.',
  'URL': 'https://ww3.lectulandia.com/book/fabulas-mitologicas-en-espana-i/'},
 {'Title': 'La otra Gioconda',


In [ ]:
#PasamoS a .pkl dado que este forma permite un manejo más óptimo
#de datos complejos y de alta demensionalidad como los embeddings

books_data.to_pickle('books_data.pkl')